In [ ]:
#General EDA questions
# what is it? graphical output and descriptive statistics for our data

#ABOUT THE DATA
# How many sites per country
# how many sites per district
# do I need to break into even more specific site groupings

# How many horizon layers per site
# depth range of horizon layers
# classify depth by its soil horizon designation

# generalize horizon labels --find out if any python package/function can set horizon label basd

#crosstabulate
## soil designation (taxonomic group and particle size)

#Pattern matching
## variation of particle size classes are within a specific subgroup (like a specific soil horizon)
## pattern match on taxonomic group, geomorphology or parent material

# filtering

# common landforms in area, commanalities in climate, 
    ## parent material, taxonomic history, geomorphic data, ecological site history (?) useful

